## Domácí úkol - Batoh

Za domácí úkol budete mít vyřešit pomocí evolučního algoritmu problém batohu. Ten spočívá v tom, že máme batoh kapacity K a N předmětů, každý s cenou c<sub>i</sub> a objemem v<sub>i</sub> a chceme vybrat takové věci, abychom maximalizovali zisk a zároveň abychom nepřekročili kapacitu batohu. 

Vstupní data máte ve složce *domaci_ukol_data*. Obsahuje čtyři soubory s daty a dva s výsledky. Na první řádce souboru s daty je vždy počet předmětů a kapacita batohu oddělené mezerou, každý další následující řádek obsahuje cenu a objem předmětu taktéž oddělené mezerou. První dva soubory slouží pro snažší odladění evolučního algoritmu a obsahují i k sobě extra soubory s optimálním řešením. Na dalších dvou máte za úkol algoritmus pustit a výsledky na nich naměřené mi poslat. 

Napište tedy nějaký svůj evoluční algoritmus, který bude řešit problém batohu a pusťte ho na vstupních datech. Svůj kód, popis evolučního algoritmu (zvolené evoluční operátory, kódování jedince, atd.) a rozbor výsledků, včetně nejlepšího dosaženého skóre i s jejich odůvodněním mi pošlete emailem do stanoveného deadline.  Pro sepsání popisu vašeho evolučního algoritmu, parametrů evoluce, zvolené reprezentace jedince a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/04_spojita_reprezentace/DU1_evolucni_algoritmy.pdf).

##### Importy

In [1]:
import array
import random
import numpy as np
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import my_algorithms

Inicializácia random itemov a veľkosť batohu

In [2]:
# random.seed(64)
random.seed()
items_count = 20

weight_bounds = (10,200)
price_bounds = (10,200)

weights = random.sample(range(weight_bounds[0],weight_bounds[1]), items_count)
prices = random.sample(range(price_bounds[0],price_bounds[1]), items_count)

max_weight = 1000

Načítanie batohu zo súboru

In [3]:
#import weights and prices from txt file

def load_input_data(file_path):

    weights = []
    prices = []

    with open(file_path) as file:
        line_split = file.readline().split(" ")
        items_count = int(line_split[0])
        max_weight = int(line_split[1])
        for i in range(items_count):
            line_split = file.readline().split(" ")
            prices.append(int(line_split[0]))
            weights.append(int(line_split[1]))

    return weights, prices, max_weight

### Sedond try - permutation individual

max fitness - berieme itemy v poradí permutácie, kým nenaplníme batoh. Ostatné neberieme

In [4]:
def fitness_batoh_perm(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight > max_weight:
            return total_price,
        total_price += prices[i]
    return total_price,

In [5]:
def fitness_batoh_perm_2(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight <= max_weight:
            total_price += prices[i]
        else:
            total_weight -= weights[i]
    return total_price,

In [6]:
def mut_transposition(individual, indv_size, transposition_count):
    count = random.randint(1, transposition_count)

    for i in range(count):
        pos_1 = random.randint(0, indv_size - 1)
        pos_2 = random.randint(0, indv_size - 1)

        individual[pos_1], individual[pos_2] = individual[pos_2], individual[pos_1]

    return individual,

In [12]:
weights, prices, max_weight = load_input_data(file_path = "./domaci_ukol_data/input_data_1000.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(indv_size), indv_size)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_batoh_perm)

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mut_transposition, indv_size=indv_size, transposition_count = 10)
toolbox.register("select", tools.selRoulette)
# toolbox.register("select", tools.selTournament, tournsize = 3)

In [13]:
POP_SIZE = 2000
GEN_COUNT = 5000


pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = my_algorithms.eaSimple(pop, toolbox, mu = 2000, cxpb=0.7, mutpb=0.3, ngen=GEN_COUNT,
                               stats=stats, halloffame=hof, verbose=True)

print(hof)
print(fitness_batoh_perm(hof[0]))

gen	nevals	avg    	std    	min 	max 
0  	2000  	4687.58	1227.13	1470	9606
1  	1610  	5488.43	1399.43	1877	10058
2  	1580  	6073.19	1511   	1818	10058
3  	1591  	6481.7 	1562.71	2846	10888
4  	1583  	6948.77	1666.36	2305	12071
5  	1576  	7358.27	1831.34	1638	12833
6  	1613  	7769.36	1959.01	1948	12833
7  	1591  	8319.55	2136.25	1948	12833
8  	1590  	9133.62	2295.69	2566	13668
9  	1618  	9972.76	2316.51	2155	13687
10 	1550  	10637.8	2207.67	3096	14189
11 	1582  	11306.5	2046.08	3504	14271
12 	1543  	11745.9	1830.83	4231	14971
13 	1591  	12083.8	1723.37	4919	15284
14 	1587  	12399.1	1662.98	4744	15926
15 	1584  	12590.8	1662.44	5629	15926
16 	1599  	12751.1	1694.76	5540	15926
17 	1582  	12833.6	1745.09	6477	16166
18 	1585  	13005.6	1853.64	6662	16166
19 	1575  	13260.6	1924.77	5560	16660
20 	1580  	13464  	1962.82	5560	16660
21 	1595  	13526.7	2044.01	6031	16782
22 	1567  	13704.9	2174.82	6031	16999
23 	1544  	14055.5	2071.97	7031	16999
24 	1522  	14157.4	2104.95	6734	17269
25 	1592  	143

In [9]:
print(hof)
print(fitness_batoh_perm(hof[0]))

[[25, 30, 736, 845, 599, 708, 480, 669, 48, 421, 986, 732, 657, 37, 549, 822, 23, 886, 612, 60, 373, 610, 32, 215, 273, 770, 13, 855, 824, 494, 469, 703, 347, 379, 987, 914, 446, 821, 419, 254, 476, 10, 121, 603, 786, 216, 945, 362, 146, 493, 134, 836, 743, 281, 573, 830, 967, 737, 426, 473, 53, 849, 382, 992, 6, 38, 592, 989, 249, 937, 334, 137, 12, 44, 952, 218, 80, 831, 28, 794, 972, 106, 860, 630, 981, 876, 393, 727, 204, 231, 990, 558, 568, 969, 792, 247, 423, 456, 781, 679, 851, 20, 438, 957, 763, 232, 642, 896, 884, 960, 496, 597, 713, 734, 445, 759, 513, 893, 283, 324, 300, 237, 182, 598, 410, 936, 129, 634, 366, 647, 139, 214, 535, 412, 151, 959, 320, 400, 489, 397, 819, 817, 207, 673, 225, 719, 602, 335, 988, 112, 889, 816, 571, 689, 979, 413, 749, 126, 726, 372, 183, 785, 510, 488, 14, 582, 683, 252, 721, 814, 793, 345, 728, 26, 188, 154, 433, 346, 81, 47, 55, 286, 177, 930, 545, 461, 569, 292, 271, 287, 178, 660, 797, 897, 576, 887, 248, 521, 620, 430, 586, 352, 448, 169, 6

#### first try - binárny jedinci

max fitness (cena ju zvyšuje a váha prekročená cez max nosnosť batohu prináša postihy)

In [ ]:
def fitness_batoh_bin(individual):
    total_price = np.sum(np.array(individual) @ np.array(prices))
    total_weight = np.sum(np.array(individual) @ np.array(weights))

    if total_weight <= max_weight:
        return total_price,
    else:
        return 1,

Vytvorenie prostredia (knižnica Deap)

In [ ]:
weights, prices = load_input_data(file_path = "./domaci_ukol_data/input_data_100.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, indv_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_batoh_bin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selRoulette)

Final part

In [ ]:
pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=200, 
                               stats=stats, halloffame=hof, verbose=True)